In [1]:
#importing the required modules

import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from __future__ import print_function
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from keras.models import Sequential
from keras.layers import Dense
from tensorflow import keras
import keras.backend as K
from itertools import product


In [2]:
# Extracting the data from the csv file and removing the outliers and then splitting into train and test sets

Z1 = pd.read_csv('clr-data.csv')
Z1 = Z1[Z1['FNC']<1e6]
Z = Z1[Z1['FNC']>0]
X = Z.drop(['Status'],axis=1)
Y = Z['Status']
X_temp_train, X_temp_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=1)
FNC_train = X_temp_train['FNC']
FNC_test = X_temp_test['FNC']
X_train = X_temp_train.drop(['FNC'],axis=1)
X_test = X_temp_test.drop(['FNC'],axis=1)

In [3]:
#function for standardizing the data using mean and standard deviation

def standardize(train, test):
    mean = np.mean(train, axis=0)
    std = np.std(train, axis=0)+0.000001
    X_train = (train - mean) / std
    X_test = (test - mean) /std
    return X_train, X_test

In [4]:
X_train, X_test = standardize(X_train,X_test)

In [5]:
# setting the costs
cost_TN = 0
cost_TP = 150
cost_FP = 150

In [6]:
# passing the false negative cost for each row will require merging the cost into the output 

def create_y_input(y_train, c_FN):
    y_str = pd.Series(y_train).reset_index(drop=True).apply(lambda x: str(int(x)))
    c_FN_str = pd.Series(c_FN).reset_index(drop=True).apply(lambda x: '0'*(6-len(str(int(x)))) + str(int(x)))
    return y_str + '.' + c_FN_str

In [7]:
# creating the customised loss function

def custom_loss2():
    def loss_function(y_input, y_pred):
        y_true = K.round(y_input)
        c_FN = (y_input - y_true) * 1e6
        cost = (y_true * (1-y_pred) * c_FN) + (y_true * (y_pred) * cost_TP) + ((1 - y_true) * ( y_pred) * cost_FP) +  ( (1 - y_true) * (1-y_pred) * cost_TN)
        return K.mean(cost, axis=-1)
    return loss_function

In [8]:
#Creating the logistic regression model

def create_model():
    model = Sequential()
    model.add(Dense(1, activation='sigmoid'))
    return model

In [9]:
#Creating and compiling the model 
model = create_model()
model.compile(optimizer='adam', loss=custom_loss2())

In [10]:
#Creating customised output by merging the False Negative Cost in the output
y_train = create_y_input(Y_train,FNC_train)
y_train = np.float32(y_train)
model.fit(X_train, y_train, batch_size=128, epochs=2, verbose=1)

Epoch 1/2

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

643/643 [==============================] - 1s 961us/step - loss: 174.7203
Epoch 2/2
643/643 [==============================] - 1s 1ms/step - loss: 101.5322


In [11]:
#Obtaining the result
y_pred = model.predict(X_test)

In [12]:
#Analysis of the result
print("accuracy =",accuracy_score(Y_test,y_pred.round()))
print("precision =",average_precision_score(Y_test, y_pred.round()))
print("recall =",recall_score(Y_test, y_pred.round()))
print("f1_score =",f1_score(Y_test, y_pred.round()))

accuracy = 0.7724633742619266
precision = 0.44137591726105907
recall = 0.7282597456312687
f1_score = 0.6046531593406593


In [13]:
#computing the savings cost
offset = len(Y_train)
cost_max=0
costs=0
for i in range(len(Y_test)):
    if(Y_test.iloc[i]==1):
        cost_max = cost_max + FNC_test.iloc[i]
for i in range(len(Y_test)):
    if(Y_test.iloc[i]==1):
        if(y_pred[i].round()==1):
            costs=costs+cost_TP
        else:
            costs=costs+FNC_test.iloc[i]
    else:
        if(y_pred[i].round()==1):
            costs=costs+cost_FP
        else:
            costs=costs+cost_TN
            
print("cost savings =",1-(costs/cost_max))

cost savings = 0.7137320615008198
